# Intraday Simulator — demo
This notebook demonstrates `simulate_trades` from `simulator.py`. Keep core logic in `simulator.py` and use this notebook for examples and visualisation.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from simulator import simulate_trades

In [ ]:
sim = simulate_trades(dt = 1.0, seed = 67) # seed = 67

# Access trade-level data
trade_times = sim['trade_times']
trade_prices = sim['prices']
volumes = sim['volumes']

# Access grid-level data
midprices = sim['midprice_grid']
grid_sigma = sim['grid_sigma']
times_grid = sim['times_grid']

# convert to dataframe for easy storage
data = {
    'time' : trade_times,
    'price' : trade_prices,
    'volume' : volumes}
df = pd.DataFrame(data)

In [ ]:
# showcase 1, here we consider only the first 60 seconds after market open for clear demonstration
# The blue line is the midprice path generated by our GARCH model
# each red dot represent a simulated trade (with time and price), which as expected, follows the blue line closely with some noise.

trade_num = len(trade_times[trade_times < 60]) # number of trades within the first 60 seconds

plt.figure(figsize=(10,4))
plt.plot(times_grid[:61], midprices[:61], linewidth=0.2, label='midprices')
plt.scatter(trade_times[:trade_num], trade_prices[:trade_num], c = 'red', s=0.2, alpha=0.6, label='trades (tick-rounded)')
plt.xlabel('seconds from start')
plt.ylabel('price')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# showcase 2
# the dark blue line is the whole-day midprice path (you can plot the trade prices instead, just looks a bit too wobble)
# the light blue bars are the minutely volumes

df['minute'] = (df['time'] // 60).astype(int) # find trade time in minutes

minute_vol = df.groupby('minute')['volume'].sum()  # sum all trade volumes in each minute (minutely volumes)             

fig, ax1 = plt.subplots(figsize=(12,4))
ax1.plot(times_grid, midprices, linewidth = 0.6, label = 'midprices')
ax1.set_xlabel('seconds from start')
ax1.set_ylabel('price')

ax2 = ax1.twinx()
ax2.bar(np.arange(len(minute_vol)) * 60, minute_vol, width = 60.0, alpha=0.35,
        align = 'edge', label = 'minutely volumes')
ax2.set_ylabel('volume')

# combined legend from both axes
h1, l1 = ax1.get_legend_handles_labels()
h2, l2 = ax2.get_legend_handles_labels()
ax1.legend(h1 + h2, l1 + l2, loc='upper left')

plt.tight_layout()
plt.show()
